In [1]:
import numpy as np
import imutils
import cv2
from matplotlib import pyplot as plt
import pandas as pd

In [2]:
def getContours(image, min_area=100, max_area=1000, min_corners=4):
    # Chuyển đổi sang ảnh xám
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Làm mờ ảnh bằng Gaussian blur
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
    # Phát hiện cạnh bằng Canny
    edges = cv2.Canny(blurred_image, 100, 500)
    # Làm dày các cạnh bằng phép Dilation
    kernel = np.ones((5, 5), np.uint8)
    dilated = cv2.dilate(edges, kernel, iterations=2)
    # Làm mỏng các cạnh bằng phép Erosion
    eroded = cv2.erode(dilated, kernel, iterations=1)
    # Tìm các contours trong ảnh
    contours, _ = cv2.findContours(eroded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Lọc các contours hợp lệ
    filtered_contours = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < min_area or area > max_area:
            continue
        perimeter = cv2.arcLength(contour, True)
        if perimeter == 0:
            continue
        # Tính số lượng cạnh của đối tượng
        approx = cv2.approxPolyDP(contour, 0.02 * perimeter, True)
        num_corners = len(approx)
        if num_corners >= min_corners:
            filtered_contours.append(contour)
    # Sắp xếp các contours theo diện tích (từ lớn đến nhỏ)
    final_contours = sorted(filtered_contours, key=cv2.contourArea, reverse=True)
    # Vẽ các contours lên ảnh
    img = image.copy()
    cv2.drawContours(img, final_contours, -1, (0, 255, 0), 2)

    return img, final_contours

In [3]:
def wrapImage(img, points, widthImg, heightImg, pad=0):

    # Chuyển đổi các điểm xác định thành numpy array
    points = np.array(points, dtype=np.float32)

    # Xác định các điểm gốc (điểm chuẩn để biến đổi hình học)
    pts_dst = np.array([
        [0, 0],
        [widthImg - 1, 0],
        [widthImg - 1, heightImg - 1],
        [0, heightImg - 1]
    ], dtype=np.float32)

    # Tính ma trận biến đổi hình học (homography)
    matrix, _ = cv2.findHomography(points, pts_dst)

    # Thực hiện phép biến đổi hình học
    warped_image = cv2.warpPerspective(img, matrix, (widthImg, heightImg))

    # Thêm padding nếu cần
    if pad > 0:
        padded_image = cv2.copyMakeBorder(warped_image, pad, pad, pad, pad, cv2.BORDER_CONSTANT, value=[0, 0, 0])
        return padded_image

    return warped_image

In [4]:
def get4Contour(contours):
    # Tính trung bình tọa độ của các điểm
    center = np.mean(contours, axis=0).reshape(-1, 2)

    # Phân chia các điểm thành hai nhóm: trên và dưới
    top_points = [pt for pt in contours if pt[0][1] < center[0][1]]
    bottom_points = [pt for pt in contours if pt[0][1] >= center[0][1]]

    # Xác định các điểm góc trên và dưới
    top_left = min(top_points, key=lambda p: p[0][0])
    top_right = max(top_points, key=lambda p: p[0][0])
    bottom_left = min(bottom_points, key=lambda p: p[0][0])
    bottom_right = max(bottom_points, key=lambda p: p[0][0])

    # Trả về các điểm góc đã được sắp xếp
    sorted_points = np.array([top_left[0], top_right[0], bottom_right[0], bottom_left[0]], dtype=np.float32)

    return sorted_points

In [5]:
def per_width():
    per_width = [0.876, 0.94]
    per_height = (0.0975, 0.298)
    return per_width, per_height

def per_width_1():
    per_width = (0.727, 0.845)
    per_height = (0.0975, 0.298)
    return per_width, per_height

def per_width_2():
    per_width = (0.055, 0.925)
    per_height = (0.326, 0.945)
    return per_width, per_height

In [6]:
def findFullAnswerSheet(pathImage, width =1830, height =2560):
  img = cv2.imread(pathImage)
  _, countours = getContours(img) #300000
  points_test_paper = get4Contour(countours[0][2])
  return wrapImage(img, points_test_paper, width, height)

def get_test_code_image(image):
    height, width, channels = image.shape
    per_width = [0.876, 0.94]
    per_height = (0.0975, 0.298)
    return image[int(per_height[0]*height):int(per_height[1]*height),\
                 int(per_width[0]*width):int(per_width[1]*width)]

def get_student_code_image(image):
    height, width, channels = image.shape
    per_width = (0.727, 0.845)
    per_height = (0.0975, 0.298)
    return image[int(per_height[0]*height):int(per_height[1]*height),\
                 int(per_width[0]*width):int(per_width[1]*width)]

def get_sheet_ans_image(image):
    height, width, channels = image.shape
    per_height = (0.325, 0.945)
    per_width = (0.04, 0.942)
    full_sheet = image[int(per_height[0]*height):int(per_height[1]*height),\
                 int(per_width[0]*width):int(per_width[1]*width)]
    return full_sheet

def get_part_sheet_ans_image(image,dis=10):
    height, width, channels = image.shape
    dis = 0.03562 * width
    A, B = image[:,:int(round((width+dis)/2 - dis,0))], image[:,int(round((width+dis)/2,0)):]
    A1,A2 = A[:,:int(round((A.shape[1]+dis)/2 - dis,0))], A[:,int(round((A.shape[1]+dis)/2,0)):]
    B1,B2 = B[:,:int(round((B.shape[1]+dis)/2 - dis,0))], B[:,int(round((B.shape[1]+dis)/2,0)):]
    return [A1,A2,B1,B2]

# In4 code

In [7]:
def get_In4code(img, num_columns, num_rows, thresh_value=150):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, thresh_value, 255, cv2.THRESH_BINARY)
    
    column_width = thresh.shape[1] // num_columns
    row_idx = [int(i * (thresh.shape[0] / num_rows)) for i in range(num_rows + 1)]
    
    code = ""
    for i in range(num_columns):
        col = thresh[:, i * column_width:(i + 1) * column_width]
        min = float('inf')
        num = None
        for j in range(num_rows):
            s, e = row_idx[j], row_idx[j + 1]
            mean = np.mean(col[s:e,:])
            if mean < min:
                min = mean
                num = j
        
        code += str(num)
    # else:
    #     code += '-'
    return code

# blobs detector algorithm

In [31]:
def get_kp(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur_img = cv2.GaussianBlur(gray_img, (5,5), 0)
    # Cài đặt thông số cho bộ phát hiện blob
    params = cv2.SimpleBlobDetector_Params()

    # Thay đổi các ngưỡng (threshold)
    params.minThreshold = 10
    params.maxThreshold = 150

    # Lọc theo diện tích (Area)
    params.filterByArea = True
    params.minArea = 100  # Điều chỉnh để phù hợp với kích thước các chấm đen trên phiếu
    params.maxArea = 1000

    # Lọc theo độ tròn (Circularity)
    params.filterByCircularity = True
    params.minCircularity = 0.7  # Những chấm đen có hình tròn

    # Lọc theo độ lồi (Convexity)
    params.filterByConvexity = True
    params.minConvexity = 0.87

    # Lọc theo độ quán tính (Inertia)
    params.filterByInertia = True
    params.minInertiaRatio = 0.01

    # Tạo bộ phát hiện blob với các tham số đã cài đặt
    detector = cv2.SimpleBlobDetector_create(params)

    # Phát hiện các blob
    keypoints = detector.detect(gray_img)
    im_with_keypoints = cv2.drawKeypoints(gray_img, keypoints, np.array([]), (0, 255, 0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
#     print(keypoints[0].pt)
    #Hiển thị hình ảnh đã phát hiện blob
#     plt.figure(figsize=(10, 10))
#     plt.imshow(im_with_keypoints, cmap='gray')
#     plt.title("Phát hiện các chấm đen trong ảnh")
#     plt.axis('off')  # Ẩn trục để hiển thị tốt hơn
#     plt.show()
    return keypoints
def show_code_detected_blobs(image, keypoints):
    """Hiển thị blobs đã phát hiện trên ảnh"""
    im_with_keypoints = cv2.drawKeypoints(image, keypoints, np.array([]),
                                          (255, 0, 0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    # cv2.imshow(im_with_keypoints)
    # cv2.waitKey(0)
    return im_with_keypoints
    
def get_ans(image_ans, key, thresh_value=150, top_offset=9, bottom_offset=7, left_offset=43, right_offset=20):
    gray = image_ans
    column_width = gray.shape[1] // 4  # 4 cột cho các đáp án A, B, C, D
    row_height = gray.shape[0] // 6  # 6 hàng cho các câu hỏi
    char_ans = ['A', 'B', 'C', 'D']
    answers = []
    question_index = 0  # Đếm thứ tự câu hỏi
    flag = False
    for col in range(4):
        # Tách từng cột đáp án
        column_img = gray[:, col * column_width:(col + 1) * column_width]
        for row in range(6):
            # Tách từng hàng câu hỏi
            start_y = row * row_height
            end_y = (row + 1) * row_height
            row_img = column_img[start_y:end_y, left_offset:-right_offset]
            row_img = row_img[top_offset:-bottom_offset, :]  # Cắt phần trên của hàng
            # Chia nhỏ từng câu hỏi trong hàng
            question_height = row_img.shape[0] // 5
            for q in range(5):
                question_img = row_img[q * question_height:(q + 1) * question_height, :]
                answer_img = question_img # Lấy phần đáp án
                bubble_width = answer_img.shape[1] // 4  # 4 lựa chọn A, B, C, D
                # Phát hiện các blob (chấm đen) trong đáp án
                blobs = get_kp(answer_img)
                # Nếu phát hiện có blob trong phần đáp án nào đó
                if blobs:
                    for blob in blobs:
                        # Tìm đáp án nào được chọn dựa vào vị trí của blob
                        for i in range(4):
                            if blob.pt[0] >= i * bubble_width and blob.pt[0] < (i + 1) * bubble_width:
                                ans = char_ans[i]
                                answers.append(ans)
                else:
                    answers.append('-')
                question_index += 1
                if question_index == len(key):
                    flag = True
                    break
            if flag:
                break
        if flag:
            break
    return answers

In [32]:
img = Image.open('test_case1.png')
img_a = np.array(img)
ans_img = get_sheet_ans_image(img_a)
ans = get_ans(ans_img, ans_keys)
ans

['A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 '-',
 'A',
 'A',
 'A',
 'A']

In [34]:
def read_image(image_path):
    img = cv2.imread(image_path)
    return img

In [35]:
def read_answer_key(answer_key_path):
    key = pd.read_excel(answer_key_path)
    return key['Answer']

In [36]:
def get_score(ans, key):
    score=0
    lst_false=[]
    true_question=0
    for i in range(len(key)):
        if ans[i] == key[i]:
            score+=1
        else:
            lst_false.append(i+1)
    percent=f"{score}/{len(key)}"
    return percent

In [37]:
answer_positions = {
    'A': (100, 200),  # Tọa độ của ô A
    'B': (200, 200),  # Tọa độ của ô B
    'C': (300, 200),  # Tọa độ của ô C
    'D': (400, 200)   # Tọa độ của ô D
}

def detect_keypoints_from_key(image, correct_answers, student_answers, box_coords):
    """
    Vẽ các đáp án lên ảnh để so sánh với đáp án của thí sinh.

    Parameters:
    - image: Ảnh chứa các câu hỏi và đáp án (numpy array)
    - correct_answers: Mảng các đáp án chính xác (list)
    - student_answers: Mảng các đáp án của thí sinh (list)
    - box_coords: Danh sách các tọa độ của các ô đáp án trên ảnh [(x1, y1, x2, y2), ...]
    """
    
    # Tạo bản sao của ảnh để vẽ lên
    output_image = image.copy()
    
    # Đặt màu sắc cho các đáp án
    color_correct = (0, 255, 0)  # Màu xanh lá cho đáp án đúng
    color_incorrect = (0, 0, 255)  # Màu đỏ cho đáp án sai
    
    # Kiểm tra số lượng đáp án và tọa độ ô đáp án có khớp không
    if len(correct_answers) != len(student_answers) or len(box_coords) != len(student_answers):
        raise ValueError("Số lượng đáp án và tọa độ ô đáp án không khớp.")
    
    # Vẽ các đáp án lên ảnh
    for i, (box, correct_answer, student_answer) in enumerate(zip(box_coords, correct_answers, student_answers)):
        x1, y1, x2, y2 = box
        
        # Vẽ khung cho ô đáp án
        cv2.rectangle(output_image, (x1, y1), (x2, y2), (255, 255, 255), 2)
        
        # Xác định màu vẽ đáp án
        color = color_correct if correct_answer == student_answer else color_incorrect
        
        # Vẽ đáp án của thí sinh
        cv2.putText(output_image, f"Student: {student_answer}", (x1 + 10, y1 + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(output_image, f"Correct: {correct_answer}", (x1 + 10, y1 + 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
    
    return output_image


In [38]:
def get_result_image(image):
        keypoints = get_kp(image)
        result_img = show_code_detected_blobs(image, keypoints)
        return result_img

# Giao diện

In [41]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import cv2
import numpy as np

def upload_image():
    global filepath
    filepath = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png")])
    if filepath:
        img = Image.open(filepath)
        img = img.resize((550, 740))  # Điều chỉnh kích thước hình ảnh cho vừa khung
        img = ImageTk.PhotoImage(img)

        image_label.config(image=img)
        image_label.image = img

        # Kích hoạt nút chấm điểm sau khi ảnh được tải lên
        grade_button.config(state=tk.NORMAL)
def upload_key():
    global filekeypath
    filekeypath = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx")])
    if filekeypath:
        try:
            # Đọc dữ liệu từ tệp Excel
            df = pd.read_excel(filekeypath)
            
            # Giả sử rằng tệp Excel có cột 'Question' và 'Answer'
            # Thay đổi tên cột và cách đọc dữ liệu theo cấu trúc tệp của bạn
            # questions = df['Question'].tolist()
            answers = df['Answer'].tolist()
            
            # Kiểm tra dữ liệu đã được tải
            if not answers:
                messagebox.showerror("Lỗi", "Tệp Excel không chứa dữ liệu hợp lệ.")
                return None
            
            # Trả về dữ liệu đáp án chính xác dưới dạng dict
            answer_key = answers
            
            # Cập nhật label với tên tệp
            file_name = filekeypath.split('/')[-1]  # Lấy tên tệp từ đường dẫn
            file_name_label.config(text=f"Tệp đã chọn: {file_name}")
            # grade_button.config(state=tk.NORMAL)
            # return answer_key

        except Exception as e:
            messagebox.showerror("Lỗi", f"Đã xảy ra lỗi khi xử lý tệp Excel: {e}")
            return None
    else:
        messagebox.showinfo("Thông báo", "Chưa chọn tệp Excel.")
        return None
ans_keys = ['A']*50    
def grade_image():
    if filepath and filekeypath:
        img = Image.open(filepath)
        img_a = np.array(img)
        # Các hàm cần thiết để xử lý ảnh
        sbd_img = get_student_code_image(img_a)
        test_img = get_test_code_image(img_a)
        ans_img = get_sheet_ans_image(img_a)
        
        # ans_keys = upload_key()
        ans = get_ans(ans_img, ans_keys)
        
        student_code = get_In4code(sbd_img, 6, 10)
        test_code = get_In4code(test_img, 3, 10)
        score = get_score(ans, ans_keys)
        result_image = get_result_image(img_a)

        # Cập nhật thông tin mã thí sinh, mã đề thi, điểm số
        info_label.config(text=f"Mã thí sinh: {student_code}\nMã đề thi: {test_code}\nĐiểm số: {score}")

        # Hiển thị ảnh kết quả chấm điểm
        # cv2.imwrite("graded_image.jpg", result_image)
        # graded_img = Image.open("graded_image.jpg")
        grade_image = result_image
        # graded_img = graded_img.resize((550, 740))  # Điều chỉnh kích thước hình ảnh cho vừa khung
        graded_img = ImageTk.PhotoImage(graded_img)

        image_label.config(image=graded_img)
        image_label.image = graded_img
    else:
        messagebox.showerror("Lỗi", "Không có ảnh nào được chọn.")

# Tạo cửa sổ chính Tkinter
root = tk.Tk()
root.title("Ứng dụng Chấm Điểm")

# Khung bên trái để chứa các nút và thông tin
left_frame = tk.Frame(root)
left_frame.pack(side=tk.LEFT, padx=10, pady=10, fill=tk.BOTH, expand=True)

# Khung bên phải để hiển thị hình ảnh
right_frame = tk.Frame(root)
right_frame.pack(side=tk.RIGHT, padx=10, pady=10, fill=tk.BOTH, expand=True)

# Nút chọn file ảnh
upload_button = tk.Button(left_frame, text="Chọn Ảnh", command=upload_image)
upload_button.pack(pady=10)

# Nút chọn file csv
upload_button = tk.Button(left_frame, text="Chọn File Đáp Án", command=upload_key)
upload_button.pack(pady=10)

# Label để hiển thị tên tệp
file_name_label = tk.Label(left_frame, text="Tệp đã chọn: Chưa có tệp")
file_name_label.pack(pady=10)

# Nút chấm điểm
grade_button = tk.Button(left_frame, text="Chấm Điểm", command=grade_image, state=tk.DISABLED)
grade_button.pack(pady=10)

# Thông tin mã thí sinh, mã đề thi, điểm số
info_label = tk.Label(left_frame, text="Mã thí sinh: \nMã đề thi: \nĐiểm số:", font=("Arial", 14))
info_label.pack(pady=10)

# Nhãn hiển thị hình ảnh bên phải
image_label = tk.Label(right_frame)
image_label.pack()

filepath = None
filekeypath = None

root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Nino\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Nino\AppData\Local\Temp\ipykernel_14480\3795883444.py", line 79, in grade_image
    graded_img = ImageTk.PhotoImage(graded_img)
                                    ^^^^^^^^^^
UnboundLocalError: cannot access local variable 'graded_img' where it is not associated with a value
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Nino\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Nino\AppData\Local\Temp\ipykernel_14480\3795883444.py", line 79, in grade_image
    graded_img = ImageTk.PhotoImage(graded_img)
                                    ^^^^^^^^^^
UnboundLocalError: cannot access local variable 'graded_img' where it is not associated with a value
